In [1]:
from keras.applications.inception_resnet_v2 import InceptionResNetV2
from keras.models import load_model, Model, Sequential
from keras.layers import Input, Dense
from keras.preprocessing import image
from keras.applications.inception_resnet_v2 import preprocess_input, decode_predictions

import numpy as np
from keras.utils.np_utils import to_categorical
import glob
import os

Using TensorFlow backend.


In [2]:
categories = ['0_buildings', '1_forest', '2_glacier', '3_mountain', '4_sea', '5_street']

In [3]:
train_datagen = image.ImageDataGenerator(preprocessing_function=preprocess_input,
                                         rotation_range=10,
                                         width_shift_range=0.2,
                                         height_shift_range=0.2,
                                         shear_range=0.01,
                                         zoom_range=[0.9, 1.25],
                                         horizontal_flip=True,
                                         vertical_flip=False,
                                         data_format='channels_last',
                                         brightness_range=[0.8, 1.2]
                                         )

In [4]:
train_generator = train_datagen.flow_from_directory(
    directory='C:/Datasets/scene-classification/data_augment/train/',
    target_size=(299, 299),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

Found 17034 images belonging to 6 classes.


```python
valid_datagen = image.ImageDataGenerator(rotation_range=10,
                                         width_shift_range=0.2, 
                                         height_shift_range=0.2,
                                         shear_range=0.01,
                                         zoom_range=[0.9, 1.25],
                                         horizontal_flip=True,
                                         vertical_flip=False,
                                         data_format='channels_last',
                                         brightness_range=[0.8, 1.2]
                                         )
```

```python
valid_generator = valid_datagen.flow_from_directory(
    directory='C:/Datasets/scene-classification/data/valid/',
    target_size=(299, 299),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    seed=42
)
```

In [5]:
STEP_SIZE_TRAIN = train_generator.n // train_generator.batch_size
# STEP_SIZE_VALID = valid_generator.n // valid_generator.batch_size

In [ ]:
input_tensor = Input(shape=(299, 299, 3))
base_model = InceptionResNetV2(include_top=False, weights='imagenet', input_tensor=input_tensor, input_shape=(299, 299, 3), pooling='avg', classes=1000)

In [7]:
for layer in base_model.layers[:-13]:
    layer.trainable = False

In [8]:
t = base_model(input_tensor)
output_tensor = Dense(len(categories), activation='softmax')(t)
model = Model(inputs=input_tensor, outputs=output_tensor)

In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 299, 299, 3)       0         
_________________________________________________________________
inception_resnet_v2 (Model)  (None, 1536)              54336736  
_________________________________________________________________
dense_1 (Dense)              (None, 6)                 9222      
Total params: 54,345,958
Trainable params: 4,711,398
Non-trainable params: 49,634,560
_________________________________________________________________


In [6]:
model = load_model('C:/Datasets/scene-classification/models/irnv2_v3_adam-contrain-v1-03-0.97.hdf5')

In [7]:
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint

```python
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3, decay=1e-6), metrics=['categorical_accuracy'])
```

In [8]:
file_path = 'C:/Datasets/scene-classification/models/'
model_name = 'irnv2_v3_adam-contrain-v1.1-{epoch:02d}-{categorical_accuracy:.2f}.hdf5'
checkpoint = ModelCheckpoint(file_path + model_name, monitor='categorical_accuracy', save_best_only=True, verbose=1, mode='max')

In [9]:
tb = TensorBoard(log_dir='C:/Datasets/scene-classification/logs/')

In [10]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    callbacks=[checkpoint, tb],
                    epochs=10,
                    verbose=1)

Epoch 1/10
532/532 [==============================] - 403s 757ms/step - loss: 0.0833 - categorical_accuracy: 0.9695

Epoch 00001: categorical_accuracy improved from -inf to 0.96942, saving model to C:/Datasets/scene-classification/models/irnv2_v3_adam-contrain-v1.1-01-0.97.hdf5
Epoch 2/10
532/532 [==============================] - 385s 724ms/step - loss: 0.0856 - categorical_accuracy: 0.9680

Epoch 00002: categorical_accuracy did not improve from 0.96942
Epoch 3/10
532/532 [==============================] - 393s 739ms/step - loss: 0.0825 - categorical_accuracy: 0.9688

Epoch 00003: categorical_accuracy did not improve from 0.96942
Epoch 4/10
532/532 [==============================] - 383s 719ms/step - loss: 0.0873 - categorical_accuracy: 0.9677

Epoch 00004: categorical_accuracy did not improve from 0.96942
Epoch 5/10
532/532 [==============================] - 384s 722ms/step - loss: 0.0830 - categorical_accuracy: 0.9679

Epoch 00005: categorical_accuracy did not improve from 0.96942
E

__Overfit the first model first, improve later__
<br>
Highest accuracy reached by t96% IRNv2: T92.9%
<br>
Stochastic Gradient Descent with warm restarts, train 1, get n, snapshot method, and Ensemble.
<br>
> Make confusion matrix, add class balances if required.
<br>
> Confusion matrix shows most discrepancy in glacier-mountain classification
<br>
> Make separate glacier-mountain classifier
<br>

Replace Adam with Stochastic Gradient Descent with warm restarts.
<br>
LeakyRelu / randomized leaky relu has noticeable impacts
<br>
Start testing on Google Colab.
<br>
Add more dense layers.
<br>
Train xgboost on last layer features from a variety of different classifiers.
<br>
More variation required in brightness
<br>
__Do not__ vary hue.

## To do:
Remove random rotations, I don't think it's helping
1. Change optimizer, set learning rate.
2. Find optimal batch size
3. ???

In [11]:
model_name = 'irnv2_v3_adam-contrain-v1.2-{epoch:02d}-{categorical_accuracy:.2f}.hdf5'

In [12]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    callbacks=[checkpoint, tb],
                    epochs=40,
                    verbose=1)

Epoch 1/40
532/532 [==============================] - 374s 703ms/step - loss: 0.0714 - categorical_accuracy: 0.9739

Epoch 00001: categorical_accuracy improved from 0.97341 to 0.97383, saving model to C:/Datasets/scene-classification/models/irnv2_v3_adam-contrain-v1.1-01-0.97.hdf5
Epoch 2/40
532/532 [==============================] - 372s 700ms/step - loss: 0.0802 - categorical_accuracy: 0.9709

Epoch 00002: categorical_accuracy did not improve from 0.97383
Epoch 3/40
532/532 [==============================] - 371s 698ms/step - loss: 0.0707 - categorical_accuracy: 0.9749

Epoch 00003: categorical_accuracy improved from 0.97383 to 0.97489, saving model to C:/Datasets/scene-classification/models/irnv2_v3_adam-contrain-v1.1-03-0.97.hdf5
Epoch 4/40
532/532 [==============================] - 371s 698ms/step - loss: 0.0770 - categorical_accuracy: 0.9743

Epoch 00004: categorical_accuracy did not improve from 0.97489
Epoch 5/40
532/532 [==============================] - 372s 699ms/step - loss

In [13]:
model_name = 'irnv2_v3_adam-contrain-v1.2-{epoch:02d}-{categorical_accuracy:.2f}.hdf5'
checkpoint = ModelCheckpoint(file_path + model_name, monitor='categorical_accuracy', save_best_only=True, verbose=1, mode='max')

In [14]:
model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    callbacks=[checkpoint, tb],
                    epochs=40,
                    verbose=1)

Epoch 1/40
532/532 [==============================] - 381s 717ms/step - loss: 0.0421 - categorical_accuracy: 0.9851

Epoch 00001: categorical_accuracy improved from -inf to 0.98518, saving model to C:/Datasets/scene-classification/models/irnv2_v3_adam-contrain-v1.2-01-0.99.hdf5
Epoch 2/40
532/532 [==============================] - 383s 720ms/step - loss: 0.0467 - categorical_accuracy: 0.9834

Epoch 00002: categorical_accuracy did not improve from 0.98518
Epoch 3/40
532/532 [==============================] - 380s 715ms/step - loss: 0.0465 - categorical_accuracy: 0.9838

Epoch 00003: categorical_accuracy did not improve from 0.98518
Epoch 4/40
532/532 [==============================] - 383s 720ms/step - loss: 0.0421 - categorical_accuracy: 0.9854

Epoch 00004: categorical_accuracy improved from 0.98518 to 0.98541, saving model to C:/Datasets/scene-classification/models/irnv2_v3_adam-contrain-v1.2-04-0.99.hdf5
Epoch 5/40
532/532 [==============================] - 381s 717ms/step - loss: 0